#### Model 2: tax processing within the base contract. The simplest tax process - must notify the user that there might be tax consequences.

In [107]:
import json
import web3
import time
from web3 import Web3, HTTPProvider
from solc import compile_source
from web3.contract import ConciseContract
from web3 import Web3, EthereumTesterProvider

In [108]:
# Solidity source code
contract_source_code = '''
pragma solidity ^0.4.25;

contract Base {
   
    //Tax procedures
    string public notification = "Your virtual currency is convertible. You need to assess your tax obligations.";

    function Notifier() public {
        notification = 'Your virtual currency is convertible. You need to assess your tax obligations.';
    }

    function setNotification(string _notification) public {
        notification = _notification;
    }

    function notify() view public returns (string) {
        return notification;
    }


    //Base function
    function withdraw() public {
        msg.sender.transfer(this.balance);
    }
}
'''

In [109]:
compiled_sol = compile_source(contract_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:Base']

In [110]:
# web3.py instance
w3 = Web3(Web3.EthereumTesterProvider())

In [111]:
# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [112]:
# Instantiate and deploy contract
Base = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

In [113]:
# Submit the transaction that deploys the contract
tx_hash = Base.constructor().transact()

In [114]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [115]:
# Create the contract instance with the newly-deployed address
base = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [116]:
# Display the default notificATION from the contract
print('Default contract NOTIFICATION: {}'.format(
    base.functions.notify().call()
))

Default contract NOTIFICATION: Your virtual currency is convertible. You need to assess your tax obligations.


In [117]:
def get_contract_interface(input_str, contract_source_code):
    # Solidity Compiler

    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol['<stdin>:' + input_str]

    # web3.py instance
    w3 = Web3(Web3.EthereumTesterProvider())

    # set pre-funded account as sender
    w3.eth.defaultAccount = w3.eth.accounts[0]

    # Instantiate and deploy contract
    ContractDeploy = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

    # Submit the transaction that deploys the contract
    tx_hash = ContractDeploy.constructor().transact()

    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

    # Create the contract instance with the newly-deployed address
    contract_inst = w3.eth.contract(
        address=tx_receipt.contractAddress,
        abi=contract_interface['abi'],
    )
    
    return w3, contract_inst